In [ ]:
#1
# объявление основных переменных/функций


import os
import os.path
import shutil
import cgitb, sys
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import platform


cfg = "/content/.config/"
#переменная !$dl для сокращения команды загрузки, перед использованием обязательно задать переменную пути $path
path = '/content/'
wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
dl = f"{wget} -P {path} "
wq = "wget -q -nv -t 10 --content-disposition"
lz4 = "tar -xI lz4 -f"
dvach = "https://huggingface.co/2ch/models/resolve/main/"
sf = "https://huggingface.co/utnah/safetensors/resolve/main/"
pt = "https://huggingface.co/utnah/ckpt/resolve/main/"
vae = "https://huggingface.co/utnah/vae/resolve/main/"
esrgan = "https://huggingface.co/utnah/esrgan/resolve/main/"
lora = "https://huggingface.co/utnah/lora/resolve/main/"
hyper = "https://huggingface.co/utnah/hypernetworks/resolve/main/"
dl_deps = "wget -q -t 10 --content-disposition https://huggingface.co/utnah/deps/resolve/main/"
deps = "https://huggingface.co/utnah/deps/resolve/main/"
pip = f"/usr/local/lib/python{'.'.join(platform.python_version().split('.')[:2])}/dist-packages/"
gh = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/"
dl_gh = f"wget -q -t 10 --content-disposition  {gh}"
end = '/content/.config/end'
deps_bar = f"{gh}depsprogressbar" # прогрессбар во время распаковки пакетов: loader(deps)
inprogress = f"{gh}loader_anim" # заглушка на css во время выполнения чего-либо: loader(inprogress)

m = HTML(requests.get(gh+'st').text)
ok = ""
with capture.capture_output() as stored_variables:
  %store cfg
  %store wget
  %store sf
  %store pt
  %store esrgan
  %store lora
  %store hyper
  %store dl_deps
  %store deps
  %store gh
  %store dl_gh
  %store end
  %store deps_bar
  %store inprogress
  %store m

vars_data = (cfg, wget, dl, wq, lz4, sf, pt, vae, esrgan, hyper, dl_deps, deps, pip, gh, dl_gh, end, deps_bar, inprogress, m)  # все переменные для сохранения в файл
with open(f'{cfg}variables', 'wb') as vars:
    pickle.dump(vars_data, vars)

def loader(html): # для подгрузки html/svg
  display(HTML(html, metadata=dict(isolated=True)))

def run_nb(filename):
  attempt = 0
  while not os.path.isfile(f"/content/.config/{filename}.ipynb") and end_attempt < 10:
    !wget -q https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/{filename}.ipynb -P /content/.config
    attempt += 1
  %run /content/.config/{filename}.ipynb
  !rm /content/.config/{filename}.ipynb  > /dev/null 2>&1

#простая качалка (для загрузки архива с зависимостями)
def download(url: str, fname: str, desc: str, chunk_size=1024):
  try:
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
      desc=desc,
      total=total,
      unit='iB',
      unit_scale=True,
      unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
          size = file.write(data)
          bar.update(size)
  except KeyboardInterrupt:
    print("\nзагрузка отменена")
  except Exception as error:
    print(f"загрузка не удалась:\n{error}")

def ng():
  end_attempt = 0
  cfg_attempt = 0
  while not os.path.isfile("/content/.config/end") and end_attempt < 10:
    !$dl_gh'end' -O /content/.config/end # svg-баннер после завершения
    end_attempt += 1
  while not os.path.isfile("/content/.config/cfg") and cfg_attempt < 10:
    !$dl_gh'zrpmdNELTAkW' -O /content/.config/cfg # нгрок-токены
    cfg_attempt += 1

#распаковка зависимостей (которые ставятся через pip)
def depinst():
  #download(f"{deps}d", "/content/d", "зависимости")
  #!pip uninstall -y torch torchvision xformers triton torchtext
  clear()
  loader(deps_bar)
  !$wq $deps'linux/linux' -P $cfg'linux'
  !cat $cfg'linux/linux' | xargs -P 100 -n1 $wq -P $cfg'linux'
  !sudo dpkg --force-all -i {cfg}linux/*.deb > /dev/null 2>&1
  !$lz4 {cfg}linux/hf_clip.tar.lz4 --directory=/ 
  !rm -rf $cfg'linux'
  with open(os.path.join(cfg, "deps"), "w") as dep:
    dep.write("\n".join([f"{deps}d/deps_{{:03d}}".format(i) for i in range(1, 48)]))
  !rm -rf /dev/shm/*
  !mount -o remount,size=11G,exec /dev/shm
  !cat {cfg}deps | xargs -P 100 -n1 $wq -P /dev/shm/
  !cat /dev/shm/deps_* > /dev/shm/deps.tar.lz4
  pip_trails = ["torchvision.libs", "xformers", "triton", "torchtext", "torchvision.libs", "torchvision", "torchaudio"]
  for pip_trail in pip_trails:
    pip_path = os.path.join(pip, pip_trail)
    if os.path.exists(pip_path):
      os.system(f"sudo rm -rf {pip_path}")
  dist_trails = re.compile(r'(xformers|triton|torch(text|vision|vision\.libs|audio|))-\S+.dist-info')
  for item in os.listdir(pip):
    if dist_trails.match(item):
      dist_path = os.path.join(pip, item)
      os.system(f"sudo rm -rf {dist_path}")
  !$lz4 /dev/shm/deps.tar.lz4 --overwrite-dir -C $pip
  !rm -rf /dev/shm/* {cfg}deps
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
  clear()

# определение папки A1111
def generate_random_file_names():
  return (name:=(''.join(random.choices(string.ascii_lowercase, k=60)))) and (fname:=name[:20], wname:=name[20:40]+'.py', lname:=name[40:]+'.py', f"/content/{fname}/")

def find_dir():
  root = "/content"
  for root, dirs, files in os.walk(root):
    for dir_name in dirs:
      if len(dir_name) == 20 and 'huge_rectal_prolapse_in_google_colab' in os.listdir(os.path.join(root, dir_name)):
        fpath = os.path.join(root, dir_name)+"/"
        return fpath

def flara(): # запрос в клаудфлару, пока не выдаст ссылку
  import os
  from subprocess import getoutput
  while True:
    !chmod +x /usr/bin/cloudflared
    !cloudflared tunnel --url localhost:7860 > clf.txt 2>&1 &
    !sleep 9
    getclfurl = f"grep -m2 -o 'https[^ ]*' {fpath}clf.txt | tail -n1 > clfr.txt"
    !$getclfurl
    srvs = getoutput(f'cat {fpath}clf.txt')
    if "has been created" in srvs:
      break
    !rm {fpath}clf.txt
    !rm {fpath}clfr.txt
  cflare = getoutput(f'cat {fpath}clfr.txt')

# css-файл для внедрения в виджеты
css_setup_widget = requests.get(f"{gh}css_setup_widget.html").text
def setup_css():
  display(HTML(f'''
    {css_setup_widget}
'''))

# функция воспроизведения аудио
def aloud(file):
  try:
    from google.colab import output as cou
    if file.startswith("http"):
      cou.eval_js('new Audio(file).play()')
    else:
      cou.eval_js(f'new Audio("{gh}{file}").play()')
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")


#проигрывание аудио, чтобы вкладка не убивалась
def mobile_player():
  try:
    cou.eval_js("var audio = new Audio('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/psyattack.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")
  except Exception as e:
    print(f"{e}: твой браузер не поддерживает автоматическое воспроизведение аудио")

# установка времени запуска колаба для таймера
try:
  launching
except:
  launching = int(time.time())-6
  with capture.capture_output() as stored_variables:
    %store launching

# автоматическое открытие ссылок градио и нгрок
def auto_opener():
  display(HTML('''
  <script>
  var linkOpened = false;
  function checkPage() {
    if (linkOpened) {
      return;
    }
    var links = document.getElementsByTagName("a");
    for (var i = 0; i < links.length; i++) {
      if (links[i].href.indexOf("gradio") !== -1) {
        window.open(links[i].href, '_blank');
        var ngroklink = document.querySelector('[href*="ngrok"]');
        window.open(ngroklink, '_blank');
        linkOpened = true;
        return;
      }
    }
  }
  checkPage();
  var observer = new MutationObserver(function(mutations) {
    mutations.forEach(function(mutation) {
      checkPage();
    });
  });
  observer.observe(document.body, { childList: true, subtree: true });
  </script>
  '''))
# отправка отчетов об ошибках
def ErrorReport(exception, error):
  import requests, html, re
  run_nb("d")
  def format_exception_text(text):
      if "Python" in text and "they occurred." in text:
          text = text.replace(text[text.index("Python"):text.index("they occurred.") + len("they occurred.")], "")
      if "__cause__" in text and "with_traceback = <" in text:
          text = text.replace(text[text.index("__cause__"):text.index("with_traceback = <") + len("with_traceback = <")], "")
      if "built-in method " in text:
          text = text.replace("built-in method ", "")
      if "\nThe above is a description" in text and "traceback:\n" in text:
          text = text.replace(text[text.index("\nThe above is a description"):text.index("traceback:\n") + len("traceback:\n")], "")
      text = text.lstrip('\n ')
      if "\n\n" in text:
          text = text.replace('\n\n', '', 1)
      if "with_traceback" in text and "object>" in text:
          text = (text.partition("with_traceback")[0] + text.partition("object>\n")[2])
      try:
        n = re.search(r"line (\d+), in <cell line", text).group(1)
        n_line = re.search(f'{n} .*', text).group()
        text = text.replace(f'  {n_line}', f'→ {n_line}')
        return text
      except:
        return text
  response = requests.post(f"{get_text('pbbxa=@@ixq$bmtmoziu$wzo@jwb6213978347=IINj50Z69qQarG3nLynFgL7IjjdGx5tlZSM@amvlUmaaiom')}", json={'chat_id': get_num('6556105208883'),'text': f"❗️ Ура, выпал пролапс: <b>{exception}</b>\n<code>{html.escape(format_exception_text(error))}</code>😄\n@unvalidable",'parse_mode': 'HTML'}, headers={'Content-type': 'application/json'})
  if response.status_code != 200:
      print("❗️ не забудь сообщить об ошибке в telegram!")
 
# совет пользователям Firefox
def check_browser():
  display(HTML('''
  <script>
  var isFirefox = typeof InstallTrigger !== 'undefined';
  var isGecko = navigator.userAgent.toLowerCase().indexOf('gecko') !== -1 && navigator.userAgent.toLowerCase().indexOf('webkit') === -1;
  var isFirefoxGecko = isFirefox || isGecko;
  var os = navigator.platform.toLowerCase();
  if (isFirefoxGecko) {
    const injectCSS = css => {
    let el = document.createElement('style');
    el.type = 'text/css';
    el.innerText = css;
    document.head.appendChild(el);
    return el;};
    injectCSS('@keyframes blink {0% {color: #ff5732;}50% {color: #eb0c0c;}100% {color: #ff5732;}}#non-chrome {position: absolute;z-index: 999;top: 0;left: 0;width: 100%;height: 100%;background-color: rgba(0, 0, 0, 0.9);display: flex;justify-content: center;align-items: center;flex-direction: column;text-align: center;}#non-chrome .browser_alarm {animation: blink 1s infinite;font-size: 1.8em }#non-chrome .expirince {font-size: 1.55em;}#non-chrome p {color: #fff }#non-chrome a {font-size: 1.55em;padding: 0 1em;color: #308df7;}#close-link {margin-top: 2em;color: #3d5d81!important;text-decoration: none;}');
    var message = document.createElement('div');
    message.id = 'non-chrome';
    document.body.appendChild(message);
    var osMessage = '';
    if (os.indexOf('win') !== -1) {
      osMessage = ' Windows';
      urls = '<p><a target="_blank" href="https://dl.google.com/update2/installers/ChromeSetup.exe">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Win/releases">Thorium</a> | <a target="_blank" href="https://github.com/Hibbiki/chromium-win64/releases/latest/">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    } else if (os.indexOf('mac') !== -1) {
      osMessage = 'macOS';
      urls = '<p><a target="_blank" href="https://dl.google.com/chrome/mac/universal/dev/googlechromedev.dmg">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Special/releases/tag/M109.0.5414.120-2">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#mac-64-bit">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/RU-ru/edge">Edge</a></p>'
    } else if (os.indexOf('linux') !== -1) {
      osMessage = 'Linux';
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium/releases/tag/M110.0.5481.178">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#linux-64-bit">Chromium</a> | <a target="_blank" href="https://packages.microsoft.com/repos/edge/pool/main/m/microsoft-edge-stable/">Edge</a></p>'
    } else {
      osMessage = os;
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com>Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    }
    var browserAlarm = '<h2 class="browser_alarm">Firefox и браузеры на его движке могут испытывать проблемы с поддержкой в '+osMessage+'</h2>';
    message.innerHTML = browserAlarm + '<p class="expirince">Для наилучшего пользовательского опыта в Google Colab установите браузер на движке Chromium:</p>' + urls + '<a href="#" id="close-link">закрыть</a>';
    var closeLink = document.getElementById('close-link');
    closeLink.addEventListener('click', function(e) {
      e.preventDefault();
      message.style.display = 'none';
    });
  }
  </script>
  '''))